In [1]:
from collections import defaultdict
import ee
import pandas as pd
import geopandas as gpd
import geemap

In [2]:
ee.Authenticate()

Enter verification code: 4/1AZEOvhVVkRGU-fI0qtUMzJ-NrNv6-t2eQqtOGcrtdIuvT4_FLWJJOKmAksk

Successfully saved authorization token.


In [2]:
ee.Initialize()

In [9]:
threeletters = {}
with open('country_codes.csv', 'r') as cc_file:
    ilines = cc_file.readlines()
    for line in ilines:
        info = line.split(',')
        threeletters[info[0].strip()] = info[1].strip()

In [3]:
urbext = ee.FeatureCollection('projects/wri-datalab/cities/urban_land_use/data/urban_extents/GHSL_BUthresh10pct_2020')

In [4]:
urbext_data = geemap.ee_to_pandas(urbext)

In [5]:
urbext_data

,rural_area,cities__in__country,Pop_2010,Country__ABC__ID,UN_RegName,urban_area,Reg_Name,Source,City__UN__ID,NB__of__Cities__Category,...,City__ID__Number,City__Local__Name,City__Name,Country__Local__Name,est_influence_distance,est_area,builtup_year,City_Name_ID_,Pop_Category,Country__Name
0,0,16,1697517,AUS,OCEANIA,7.004928e+08,Land-Rich Developed Countries (LRDC),UN Population Division,206172.0,2,...,1101,Perth,Perth,Commonwealth of Australia,2391.455370,1.289366e+09,2020,Perth,3,Australia
1,0,16,119134,AUS,OCEANIA,1.177174e+07,Land-Rich Developed Countries (LRDC),UN Population Division,206174.0,2,...,1094,Darwin,Darwin,Commonwealth of Australia,775.480918,1.355794e+08,2020,Darwin,1,Australia
2,0,19,306894,IRQ,Western Asia,4.826206e+07,Western Asia and North Africa (WANA),UN Population Division,21539.0,2,...,3484,Ar-Ram?d?,Ramadi,Republic of Iraq,504.723214,5.743255e+07,2020,Ramadi,1,Iraq
3,0,19,353549,IRQ,Western Asia,3.941541e+07,Western Asia and North Africa (WANA),UN Population Division,21531.0,2,...,3474,Ad D?w?n?yah,Diwaniyah,Republic of Iraq,537.049754,6.502503e+07,2020,Diwaniyah,1,Iraq
4,0,19,356802,IRQ,Western Asia,2.636773e+07,Western Asia and North Africa (WANA),UN Population Division,23323.0,2,...,3470,Ba�q?bah,Baqubah,Republic of Iraq,539.211826,6.554965e+07,2020,Baqubah,1,Iraq
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4224,0,7,1684180,BOL,Latin America And The Carribean,2.467147e+08,Latin America and the Caribbean (LAC),UN Population Division,20157.0,1,...,1188,Santa Cruz,Santa Cruz,Plurinational State of Bolivia,1097.668734,2.716403e+08,2020,Santa_Cruz,3,Bolivia
4225,0,7,160663,BOL,Latin America And The Carribean,1.528615e+07,Latin America and the Caribbean (LAC),UN Population Division,20155.0,1,...,1187,Potosi,Potosi,Plurinational State of Bolivia,406.710861,3.729264e+07,2020,Potosi,1,Bolivia
4226,0,7,202551,BOL,Latin America And The Carribean,2.960424e+07,Latin America and the Caribbean (LAC),UN Population Division,20159.0,1,...,1190,Tarija,Tarija,Plurinational State of Bolivia,448.539431,4.535789e+07,2020,Tarija,1,Bolivia
4227,0,7,225137,BOL,Latin America And The Carribean,4.811927e+07,Latin America and the Caribbean (LAC),UN Population Division,20154.0,1,...,1186,Oruro,Oruro,Plurinational State of Bolivia,469.029614,4.959662e+07,2020,Oruro,1,Bolivia


In [10]:
bv_2015 = ee.Image('projects/wri-datalab/SCL-Cities/GHS-Built-V_2015')
bv_2020 = ee.Image('projects/wri-datalab/SCL-Cities/GHS-Built-V_2020')

In [11]:
worldpop_2020 = ee.ImageCollection("WorldPop/GP/100m/pop").filter(ee.Filter.equals('year', 2020))
worldpop_2015 = ee.ImageCollection("WorldPop/GP/100m/pop").filter(ee.Filter.equals('year', 2015))

In [8]:
from ipywidgets import IntProgress
from IPython.display import display
f = IntProgress(min=0, max=4231)
display(f)

IntProgress(value=0, max=4231)

In [13]:
result_data = defaultdict(list)
for i in range(len(urbext_data)):
    f.value += 1
    ua = urbext_data.iloc[i]
    country_code = ua['Country__ABC__ID']
    if len(country_code) == 2:
        country_code = threeletters[country_code]
    ua_f = urbext.filter(ee.Filter.equals('City__ID__Number', int(ua['City__ID__Number'])))
    geom = ua_f.geometry()
    scale_idx = 0
    worked = False
    scale = 100
    local_bv_2020 = bv_2020.reduceRegion(ee.Reducer.sum(), geom, scale, maxPixels=1e12).getInfo()['b1']
    local_bv_2015 = bv_2015.reduceRegion(ee.Reducer.sum(), geom, scale, maxPixels=1e12).getInfo()['b1']
    
    localpop_2020_img = worldpop_2020.filter(ee.Filter.equals('country', country_code)).select('population').first()
    localpop_2020 = localpop_2020_img.reduceRegion(ee.Reducer.sum(), geom, scale, maxPixels=1e12).getInfo()['population']
    localpop_2015_img = worldpop_2015.filter(ee.Filter.equals('country', country_code)).select('population').first()
    localpop_2015 = localpop_2015_img.reduceRegion(ee.Reducer.sum(), geom, scale, maxPixels=1e12).getInfo()['population']

    result_data[ua['Country__Name']].append(( ua['City__Name'],
                                             local_bv_2015,
                                             local_bv_2020,
                                             localpop_2015,
                                             localpop_2020
                                            ))
    if i % 100 == 0:
        print('{0}/{1}'.format(i+1, len(urbext_data)))

1/4229
101/4229
201/4229
301/4229
401/4229
501/4229
601/4229
701/4229
801/4229
901/4229
1001/4229
1101/4229
1201/4229
1301/4229
1401/4229
1501/4229
1601/4229
1701/4229
1801/4229
1901/4229
2001/4229
2101/4229
2201/4229
2301/4229
2401/4229
2501/4229
2601/4229
2701/4229
2801/4229
2901/4229
3001/4229
3101/4229
3201/4229
3301/4229
3401/4229
3501/4229
3601/4229
3701/4229
3801/4229
3901/4229
4001/4229
4101/4229
4201/4229


In [15]:
results = {country: {
    'built_volume_cubicmeters_2015': sum([i[1] for i in result_data[country]]),
    'built_volume_cubicmeters_2020': sum([i[2] for i in result_data[country]]),
    'built_volume_change_cubicmeters': sum([i[2] for i in result_data[country]]) - sum([i[1] for i in result_data[country]]),
    'built_volume_changerate_relative2015': (sum([i[2] for i in result_data[country]]) - sum([i[1] for i in result_data[country]])) / sum([i[1] for i in result_data[country]]),
    'population_2015': sum([i[3] for i in result_data[country]]),
    'population_2020': sum([i[4] for i in result_data[country]]),
    'population_change': sum([i[4] for i in result_data[country]]) - sum([i[3] for i in result_data[country]]),
    'population_changerate_relative2015': (sum([i[4] for i in result_data[country]]) - sum([i[3] for i in result_data[country]])) / sum([i[3] for i in result_data[country]])
} for country in result_data}
result_df = pd.DataFrame(results).transpose()
result_df.to_csv('built_volume_changerate.csv')